# Example to Read / Write to Microsoft SQL Server with Spark

Generic JDBC Documentation: https://spark.apache.org/docs/3.1.2/sql-data-sources-jdbc.html

This approach should be similar for any JDBC database.

## mssql-cli 

This non-spark tool allows you to manage the database from the CLI. We use it here to create a database.

In [1]:
!pip install -qq mssql-cli

In [2]:
!mssql-cli -S mssql -U sa -P SU2Orange! -Q "CREATE DATABASE TEST"

Commands completed successfully.


## Spark



In [5]:
import pyspark
from pyspark.sql import SparkSession

In [6]:
# SQLSERVRER CONFIGURATION
server = "mssql"
server_str = f"jdbc:sqlserver://{server}:1433"
database_name = "master" 
username = "sa"
password = "SU2Orange!" 
url = f"{server_str};databaseName={database_name};user={username};password={password};authentication=SqlPassword;encrypt=true;trustServerCertificate=true"
print(url)

jdbc:sqlserver://mssql:1433;databaseName=master;user=sa;password=SU2Orange!;authentication=SqlPassword;encrypt=true;trustServerCertificate=true


In [7]:
# Spark init
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
      .config("spark.jars.packages","com.microsoft.sqlserver:mssql-jdbc:9.4.1.jre11")\
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ce67ad77-9e87-4e2d-87b7-3fa4696b79f1;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.microsoft.sqlserver#mssql-jdbc;9.4.1.jre11 in central
:: resolution report :: resolve 111ms :: artifacts dl 2ms
	:: modules in use:
	com.microsoft.sqlserver#mssql-jdbc;9.4.1.jre11 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-ce67ad77-9e87-4e2d-87b7-3fa4696b79f1
	confs: [default]
	0 artifacts copied, 1 already retrieved (0kB/6ms)
22/01/07 17:46:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/

In [21]:
# read local data
df = spark.read.option("multiline","true").json("/home/jovyan/datasets/json-samples/stocks.json")
df.toPandas()

,price,symbol
0,126.82,AAPL
1,3098.12,AMZN
2,251.11,FB
3,1725.05,GOOG
4,128.39,IBM
5,212.55,MSFT
6,78.00,NET
7,497.00,NFLX
8,823.80,TSLA
9,45.11,TWTR


## Writing Data

- In spark Overwrite mode, the table will be re-created each time.
- Because there is no SQL, a table key cannot be specified without using a custom schema.

In [28]:
# Write create the table, inferring a schema on write:
tablename = "stocks"
df.write.format("jdbc") \
    .mode("overwrite") \
    .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url", url) \
    .option("dbtable", tablename) \
    .save()

df.printSchema()

root
 |-- price: double (nullable = true)
 |-- symbol: string (nullable = true)



## Reading Data

- Reading data back out can be done with a SELECT statement

In [30]:
# read by query
sql = "select * from stocks where symbol like 'N%'"
stocks = spark.read.format("jdbc") \
    .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url", url) \
    .option("query", sql) \
    .load()
    
stocks.toPandas()

,price,symbol
0,78.0,NET
1,497.0,NFLX


## Writing data to an existing table

- use spark's "append" modifier to write the data.
- in typical SQL fashion, constraint failures will cause the write to rollback.

In [39]:
# we can add a stock with append mode

from pyspark.sql.types import StructType, StructField, StringType, DoubleType
data = [("GME", 136.34)]
schema = StructType([\
    StructField("symbol",StringType(),nullable=True), \
    StructField("price",DoubleType(),nullable=True), \
    ])
                     
newstock = spark.createDataFrame(data=data,schema=schema)

newstock.write.format("jdbc") \
    .mode("append") \
    .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url", url) \
    .option("dbtable", "stocks") \
    .save()

newstock.toPandas()


,symbol,price
0,GME,136.34


In [41]:
# see the new stock added
sql = "select * from stocks where symbol like 'G%'"
stocks = spark.read.format("jdbc") \
    .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url", url) \
    .option("query", sql) \
    .load()
stocks.toPandas()

,price,symbol
0,1725.05,GOOG
1,136.34,GME


## Executing Arbitrary SQL 

- Spark / JDBC is not appropriate for executing DDL or DML commands like CREATE TABLE or UPDATE
- Instead use a non-big data tool, like your typical SQL client.
- You can use `mssql-cli` as demonstrated above 

In [22]:
# truncate the table
!!mssql-cli -S mssql -U sa -P SU2Orange! -Q "TRUNCATE TABLE stocks"

['Commands completed successfully.']